## Importing Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn import LeakyReLU,ReLU,Tanh,Sigmoid,Softmax
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset,DataLoader
import torchvision.models as models
!pip install timm
import timm

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import joblib
import cv2
import os
from tqdm.notebook import tqdm
import time
import random
from PIL import Image,ImageOps
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Setting Seed

In [ ]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True 
SEED=42
seed_everything(SEED=SEED)

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Dataset Creation

In [ ]:
torchvision.datasets.Caltech101(root = 'Data', download = True)

Files already downloaded and verified


Dataset Caltech101
    Number of datapoints: 8677
    Root location: Data/caltech101
    Target type: ['category']

In [ ]:
path = []
label = []

parent = os.path.join('Data','caltech101','101_ObjectCategories')

for i,j in enumerate(os.listdir(parent)):

  category = os.path.join(parent,j)

  for k in os.listdir(category):
    path.append(os.path.join(category,k))
    label.append(i)

df = pd.DataFrame({'Image':path,'Label':label})

In [ ]:
df = df.groupby('Label').apply(lambda x: x.sample(30))

In [ ]:
df_train,df_test = train_test_split(df, test_size=0.2, random_state=SEED,stratify = df.Label)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=SEED,stratify = df_train.Label)

In [ ]:
class ImageData(Dataset):
    def __init__(self,df,transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img_path = self.df.iloc[index,0]
        labels = torch.tensor(self.df.iloc[index,1],dtype=torch.long)
        image = (Image.open(img_path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, labels

transform = transforms.Compose([transforms.Resize(size = (224,224)),transforms.ToTensor(), transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])])#

train_generator = DataLoader(ImageData(df_train,transform=transform),batch_size = 100,shuffle = True)
val_generator = DataLoader(ImageData(df_val,transform=transform),batch_size = 100,shuffle = True)
test_generator = DataLoader(ImageData(df_test,transform=transform),batch_size = 100,shuffle = True)

## Model Creation

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name = 'mobilenetv3_large_100', pretrained = True):
        super().__init__()
        self.model_name = model_name
        self.cnn = timm.create_model(self.model_name, pretrained = pretrained, num_classes = 102)

    def forward(self, x):
        x = self.cnn(x)
        return x

## Train, Validation and Test Functions

In [ ]:
def train_fn(train_loader, model, criterion, optmizer, device):
  model.train()
  
  size = len(train_loader.dataset)
  num_batches = len(train_loader)

  loss, correct = 0, 0

  for batch, (X, y) in tqdm(enumerate(train_loader)):

    start = time.time()

    device = torch.device(device)
    X, y = X.to(device), y.to(device)  

    optimizer.zero_grad()
    pred = model(X)
    loss = criterion(pred, y.long()) 
    current = batch * len(X)

    loss.backward()
    optimizer.step()

    y_pred, y_true = torch.argmax(pred, axis=1), y.long().squeeze()
    correct += (y_pred == y_true).type(torch.float).sum().item()

    end = time.time()
    time_delta = np.round(end - start, 3)

    loss, current = np.round(loss.item(), 5), batch * len(X)

  correct /= size 
  loss /= num_batches

  print(f"Train: Accuracy: {(100*correct):>0.2f}%, Avg loss: {loss:>5f} \n")

  return loss, correct


In [ ]:
def valid_fn(valid_loader, model, criterion, device):
  model.eval()

  size = len(valid_loader.dataset)
  num_batches = len(valid_loader)

  loss, correct = 0, 0


  with torch.no_grad(): 
    for batch, (X, y) in enumerate(valid_loader):

      start = time.time()

      device = torch.device(device)
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = criterion(pred, y.long().squeeze()) 
      current = batch * len(X)
      
      y_pred, y_true = torch.argmax(pred, axis=1), y.long().squeeze()
      correct += (y_pred == y_true).type(torch.float).sum().item()

      end = time.time()
      time_delta = np.round(end - start, 3)
      
      loss, current = np.round(loss.item(), 5), batch * len(X)
      
  correct /= size 
  loss /= num_batches

  print(f"Valid: Accuracy: {(100*correct):>0.2f}%, Avg loss: {loss:>5f} \n")

  return loss, correct

In [ ]:
def test_fn(test_loader, model, criterion, device):
  model.eval()

  size = len(test_loader.dataset)
  num_batches = len(test_loader)

  loss, correct = 0, 0

  with torch.no_grad(): 
    for batch, (X, y) in enumerate(test_loader):

      start = time.time()

      device = torch.device(device)
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = criterion(pred, y.long().squeeze()) 
      current = batch * len(X)
      
      y_pred, y_true = torch.argmax(pred, axis=1), y.long().squeeze()
      correct += (y_pred == y_true).type(torch.float).sum().item()

      end = time.time()
      time_delta = np.round(end - start, 3)
      
      loss, current = np.round(loss.item(), 5), batch * len(X)
     
  correct /= size 
  loss /= num_batches 

  print(f"Test: Accuracy: {(100*correct):>0.2f}%, Avg loss: {loss:>5f} \n")

  print(classification_report(y_true,y_pred))

  return loss, correct

## Model 1 - Transfer Learning

In [ ]:
start = time.time()

loss_fn = nn.CrossEntropyLoss()

device = torch.device(device)
model = Model()

for param in model.parameters():
    param.requires_grad = False

for param in model.cnn.classifier.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-3,amsgrad = False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

train_loss_history = []
train_acc_history = []
valid_loss_history = []
valid_acc_history = []
LR_history = []

best_loss = np.inf
best_epoch_loss = 0
best_acc = 0
best_epoch_acc = 0

print('Starting Training...\n')

start_train_time = time.time()

EPOCH = 10
for epoch in range(0, EPOCH):
  print(f"\n-------------------------------   Epoch {epoch + 1}   -------------------------------\n")
  start_epoch_time = time.time()

  train_loss, train_acc = train_fn(train_generator, model, loss_fn, optimizer, device)
  train_loss_history.append(train_loss)
  train_acc_history.append(train_acc)

  valid_loss, valid_acc = valid_fn(val_generator, model, loss_fn, device)
  valid_loss_history.append(valid_loss)
  valid_acc_history.append(valid_acc)


  scheduler.step(valid_loss)

  for param_group in optimizer.param_groups:
    LR_history.append(param_group['lr'])

  if valid_loss < best_loss:
    best_epoch_loss = epoch + 1
    best_loss = valid_loss
    torch.save(model.state_dict(), './' + f"Model_ep{best_epoch_loss}.pth")

  if valid_acc > best_acc:
    best_epoch_acc = epoch + 1
    best_acc = valid_acc
    torch.save(model.state_dict(), './' + f"Model_ep{best_epoch_acc}.pth")    

  end_epoch_time = time.time()
  time_delta = np.round(end_epoch_time - start_epoch_time, 3)
  print("\n\nEpoch Elapsed Time: {} s".format(time_delta))

test_fn(test_generator, model, loss_fn, device)

end_train_time = time.time()
print("\n\nTotal Elapsed Time: {} min".format(np.round((end_train_time - start_train_time)/60, 3)))
print("Done!")

Starting Training...


-------------------------------   Epoch 1   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 11.90%, Avg loss: 0.185967 

Valid: Accuracy: 33.47%, Avg loss: 0.714884 



Epoch Elapsed Time: 130.311 s

-------------------------------   Epoch 2   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 59.55%, Avg loss: 0.112360 

Valid: Accuracy: 61.02%, Avg loss: 0.471320 



Epoch Elapsed Time: 124.49 s

-------------------------------   Epoch 3   -------------------------------



...

Train: Accuracy: 99.13%, Avg loss: 0.009669 

Valid: Accuracy: 83.06%, Avg loss: 0.192636 



Epoch Elapsed Time: 121.29 s

-------------------------------   Epoch 9   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 99.39%, Avg loss: 0.009702 

Valid: Accuracy: 83.88%, Avg loss: 0.162574 



Epoch Elapsed Time: 121.847 s

-------------------------------   Epoch 10   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 99.59%, Avg loss: 0.009562 

Valid: Accuracy: 83.67%, Avg loss: 0.143920 



Epoch Elapsed Time: 120.479 s
Test: Accuracy: 83.50%, Avg loss: 0.090801 

              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         1
          21       1.00      1.00      1.00         1
          22       0.00      0.00      0.00         1
          30       1.00      1.00      1.00         1
          31       1.00      1.00      1.00         1
          51       0.00      0.00      0.00         0
          54       1.00      1.00      1.00         1
          58       0.00      0.00      0.00         0
          75       1.00      1.00      1.00         1
          85       1.00      1.00      1.00         1
          92       1.00      1.00      1.00         1
          96       1.00      1.00      1.00         1

    accuracy       

## Model 2 - Randomly initializing weights

In [17]:
start = time.time()

loss_fn = nn.CrossEntropyLoss()

device = torch.device(device)
model2 = model = Model(pretrained = False)
# model2.classifier[3] = torch.nn.Linear(in_features=1024, out_features=102)

optimizer = optim.Adam(model2.parameters(), lr=1e-3,amsgrad = False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

train_loss_history = []
train_acc_history = []
valid_loss_history = []
valid_acc_history = []
LR_history = []

best_loss = np.inf
best_epoch_loss = 0
best_acc = 0
best_epoch_acc = 0

print('Starting Training...\n')

start_train_time = time.time()

EPOCH = 10
for epoch in range(0, EPOCH):
  print(f"\n-------------------------------   Epoch {epoch + 1}   -------------------------------\n")
  start_epoch_time = time.time()

  train_loss, train_acc = train_fn(train_generator, model2, loss_fn, optimizer, device)
  train_loss_history.append(train_loss)
  train_acc_history.append(train_acc)

  valid_loss, valid_acc = valid_fn(val_generator, model2, loss_fn, device)
  valid_loss_history.append(valid_loss)
  valid_acc_history.append(valid_acc)


  scheduler.step(valid_loss)

  for param_group in optimizer.param_groups:
    LR_history.append(param_group['lr'])

  if valid_loss < best_loss:
    best_epoch_loss = epoch + 1
    best_loss = valid_loss
    torch.save(model2.state_dict(), './' + f"Model2_ep{best_epoch_loss}.pth")

  if valid_acc > best_acc:
    best_epoch_acc = epoch + 1
    best_acc = valid_acc
    torch.save(model2.state_dict(), './' + f"Model2_ep{best_epoch_acc}.pth")    

  end_epoch_time = time.time()
  time_delta = np.round(end_epoch_time - start_epoch_time, 3)
  print("\n\nEpoch Elapsed Time: {} s".format(time_delta))


test_fn(test_generator, model2, loss_fn, device)

end_train_time = time.time()

print("\n\nTotal Elapsed Time: {} min".format(np.round((end_train_time - start_train_time)/60, 3)))

print("Done!")

Starting Training...


-------------------------------   Epoch 1   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 1.33%, Avg loss: 0.227358 

Valid: Accuracy: 1.02%, Avg loss: 0.995036 



Epoch Elapsed Time: 300.973 s

-------------------------------   Epoch 2   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 4.49%, Avg loss: 0.229224 

Valid: Accuracy: 4.69%, Avg loss: 0.926176 



Epoch Elapsed Time: 283.015 s

-------------------------------   Epoch 3   -------------------------------



...

Train: Accuracy: 80.18%, Avg loss: 0.056622 

Valid: Accuracy: 10.00%, Avg loss: 1.313528 



Epoch Elapsed Time: 277.843 s

-------------------------------   Epoch 9   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 83.35%, Avg loss: 0.032853 

Valid: Accuracy: 11.02%, Avg loss: 1.335812 



Epoch Elapsed Time: 276.799 s

-------------------------------   Epoch 10   -------------------------------



0it [00:00, ?it/s]

Train: Accuracy: 88.82%, Avg loss: 0.026949 

Valid: Accuracy: 11.02%, Avg loss: 1.354808 



Epoch Elapsed Time: 276.922 s
Test: Accuracy: 12.58%, Avg loss: 0.955547 

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          23       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          38       0.00      0.00      0.00         0
          45       0.00      0.00      0.00         1
          46       0.00      0.00      0.00         0
          47       0.00      0.00      0.00         0
          50       1.00      1.00      1.00         1
          57       0.00      0.00      0.00         0
          60       0.00      0.00      0.00         1
          70       0